In [23]:
import pandas as pd

df = pd.read_csv('customer_churn.csv')
display(df.head())

,tenure,PhoneService,InternetService,MonthlyCharges,TotalCharges,Churn
0,1,No,DSL,29.85,29.85,Stayed
1,34,Yes,DSL,56.95,1889.5,Stayed
2,2,Yes,DSL,53.85,108.15,Churned
3,45,No,DSL,42.30,1840.75,Stayed
4,2,Yes,Fiber optic,70.70,151.65,Churned


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tenure           7043 non-null   int64  
 1   PhoneService     7043 non-null   object 
 2   InternetService  7043 non-null   object 
 3   MonthlyCharges   7043 non-null   float64
 4   TotalCharges     7043 non-null   object 
 5   Churn            7043 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 330.3+ KB


In [25]:
# Check for missing values
print("Missing values before handling:")
print(df.isnull().sum())

Missing values before handling:
tenure             0
PhoneService       0
InternetService    0
MonthlyCharges     0
TotalCharges       0
Churn              0
dtype: int64


In [26]:

# Convert 'TotalCharges' to numeric, coercing errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Handle missing values in 'TotalCharges' by filling with the mean
# df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)

print("\nMissing values after handling:")
print(df.isnull().sum())


Missing values after handling:
tenure              0
PhoneService        0
InternetService     0
MonthlyCharges      0
TotalCharges       11
Churn               0
dtype: int64


In [27]:
# Handle missing values in 'TotalCharges' by filling with the mean
df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)

print("\nMissing values after handling:")
print(df.isnull().sum())


Missing values after handling:
tenure             0
PhoneService       0
InternetService    0
MonthlyCharges     0
TotalCharges       0
Churn              0
dtype: int64


C:\Users\ANC\AppData\Local\Temp\ipykernel_8732\1027756482.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)


In [28]:
# df.value_counts('Churn')

In [29]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode categorical features
for column in ['PhoneService', 'InternetService', 'Churn']:
    if column in df.columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])

display(df.head())

,tenure,PhoneService,InternetService,MonthlyCharges,TotalCharges,Churn
0,1,0,0,29.85,29.85,1
1,34,1,0,56.95,1889.50,1
2,2,1,0,53.85,108.15,0
3,45,0,0,42.30,1840.75,1
4,2,1,1,70.70,151.65,0


In [30]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop('Churn', axis=1)
y = df['Churn']

# Scale numerical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (5634, 5)
X_test shape: (1409, 5)
y_train shape: (5634,)
y_test shape: (1409,)


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the ANN model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

C:\Users\ANC\miniconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 32)                  │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 737 (2.88 KB)

 Trainable params: 737 (2.88 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7360 - loss: 0.5212 - val_accuracy: 0.7728 - val_loss: 0.4569
Epoch 2/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7781 - loss: 0.4640 - val_accuracy: 0.7879 - val_loss: 0.4379
Epoch 3/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7834 - loss: 0.4552 - val_accuracy: 0.7888 - val_loss: 0.4329
Epoch 4/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7806 - loss: 0.4523 - val_accuracy: 0.7888 - val_loss: 0.4299
Epoch 5/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7826 - loss: 0.4500 - val_accuracy: 0.7933 - val_loss: 0.4299
Epoch 6/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7841 - loss: 0.4487 - val_accuracy: 0.7941 - val_loss: 0.4277
Epoch 7/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7834 - loss: 0.4486 - val_accuracy: 0.7888 - val_loss: 0.4274
Epoch 8/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7857 - loss: 0.4478 - val_accuracy: 0.

In [33]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8055 - loss: 0.4174
Test Loss: 0.4174
Test Accuracy: 0.8055


In [34]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predict on the test data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int) # Convert probabilities to binary predictions

# Calculate and display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Display classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Confusion Matrix:
[[198 175]
 [ 99 937]]

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.53      0.59       373
           1       0.84      0.90      0.87      1036

    accuracy                           0.81      1409
   macro avg       0.75      0.72      0.73      1409
weighted avg       0.80      0.81      0.80      1409



**Reasoning**:
The previous command failed because the `keras_tuner` library is not installed. Install `keras_tuner` using pip.



In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the ANN model
model = Sequential([
    (64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='sigmoid')
])


# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

C:\Users\ANC\miniconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 64)                  │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 16)                  │             528 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,992 (11.69 KB)

 Trainable params: 2,992 (11.69 KB)

 Non-trainable params: 0 (0.00 B)